In [1]:
import rospy, baxter_commander, json
import numpy as np
import tf
import time
from scipy.optimize import minimize
import transformations
import rospkg
import yaml

In [2]:
rospy.init_node('vrep_calibration')

In [3]:
rospack = rospkg.RosPack()

In [4]:
right = baxter_commander.ArmCommander('right')

Unknown tag: materialUnknown tag: materialScalar element defined multiple times: collisionScalar element defined multiple times: collision

# Defintion of Functions

### Tools

In [5]:
def random_transforms(pos_bounds, rot_bounds, nb_transforms=2):
    flat_transforms = []
    for i in range(nb_transforms):
        # add a random position within bounds
        flat_transforms += np.random.uniform(pos_bounds[0], pos_bounds[1], 3).tolist()
        # add a random quaternion
        rot = np.random.uniform(rot_bounds[0], rot_bounds[1], 4)
        flat_transforms += (rot / np.linalg.norm(rot)).tolist()
    return flat_transforms

In [6]:
def extract_transforms(flat_transforms):
    # a transform is 3 pos and 4 rot
    nb_transform = len(flat_transforms) / 7
    list_transforms = []
    for i in range(nb_transform):
        pose = []
        # extract the pose
        pose.append(flat_transforms[i * 7:i * 7 + 3])
        pose.append(flat_transforms[i * 7 + 3:i * 7 + 7])
        # append it to the list of transforms
        list_transforms.append(pose)
    return list_transforms

### Cost function for the optimization

In [7]:
def distance_cost(pose1, pose2, rot_coeff=2):
    pos_cost = 0
    # calculate position ditance
    pos_cost = np.linalg.norm(np.array(pose1[0]) - np.array(pose2[0]))
    # distance between two quaternions
    rot_cost = 1 - np.inner(pose1[1], pose2[1])**2
    return pos_cost + rot_coeff * rot_cost

In [8]:
def quaternion_cost(list_calibr, norm_coeff):
    C = 0
    for transform in list_calibr:
        # norm of a quaternion is always 1
        C += norm_coeff * abs(np.linalg.norm(transform[1]) - 1)
    return C

In [9]:
def evaluate_calibration(calibrations, tf_robot, tf_vrep):
    # first extract the transformations
    list_calibr = extract_transforms(calibrations)
    # collect the cost based on the quaternions
    cost = quaternion_cost(list_calibr, 0.5)
    # loop trough all the tf
    for i in range(len(tf_robot)):
        # compute the corresponding transformation from recorded data
        pose = transformations.multiply_transform(list_calibr[0], tf_vrep[i])
        pose = transformations.multiply_transform(pose, list_calibr[1])
        # compute the cost based on the distance
        cost += distance_cost(tf_robot[i], pose)
    return cost

## Record Points

In [10]:
points = {"tf": [], "joints":[]}
tf_robot = []
while True:
    entry = raw_input('Press Enter to record a new point or Q-Enter to exit: ')
    if entry!='':
        break
    fk = right.endpoint_pose()
    joints = baxter_commander.persistence.statetodict(right.get_current_state())
    tf_robot.append([fk[0], fk[1]])
    points["tf"].append(fk[0]+fk[1])
    points["joints"].append(joints["position"])

Press Enter to record a new point or Q-Enter to exit: 
Press Enter to record a new point or Q-Enter to exit: 
Press Enter to record a new point or Q-Enter to exit: 
Press Enter to record a new point or Q-Enter to exit: 
Press Enter to record a new point or Q-Enter to exit: 
Press Enter to record a new point or Q-Enter to exit: 
Press Enter to record a new point or Q-Enter to exit: 
Press Enter to record a new point or Q-Enter to exit: 
Press Enter to record a new point or Q-Enter to exit: 
Press Enter to record a new point or Q-Enter to exit: 
Press Enter to record a new point or Q-Enter to exit: q


In [21]:
points = {"tf": [], "joints":[]}
tf_robot = []
for i in range(10):
    goal = right.get_random_pose()
    # display only first
    right.move_to_controlled(goal, display_only=True)
    entry = raw_input('Is the trajectory correct ? y/[N] ')
    if entry == 'y':
        # now execute the trajectory
        right.move_to_controlled(goal)
        fk = right.endpoint_pose()
        joints = baxter_commander.persistence.statetodict(right.get_current_state())
        tf_robot.append([fk[0], fk[1]])
        points["tf"].append([fk[0], fk[1]])
        points["joints"].append(joints["position"])

Is the trajectory correct ? y/[N] 
Is the trajectory correct ? y/[N] y
Is the trajectory correct ? y/[N] y
Is the trajectory correct ? y/[N] 
Is the trajectory correct ? y/[N] 
Is the trajectory correct ? y/[N] 
Is the trajectory correct ? y/[N] 
Is the trajectory correct ? y/[N] 
Is the trajectory correct ? y/[N] 
Is the trajectory correct ? y/[N] 


In [11]:
with open('/tmp/robot_joints.json', 'w') as f:
    json.dump(points, f, indent=4)

## Matlab part: Get VREP coordinates with the given joint angles

In [12]:
with open('/tmp/vrep_transform.json') as f:
    data = json.load(f)
# convert the format to get transform matrices
positions = np.array(data['cartesian_XYZ'])
quaternions = np.array(data['quaternion_XYZScale'])
tf_vrep = []
for i in range(len(tf_robot)):
    pos = positions[i]
    rot = quaternions[i]
    tf_vrep.append([pos, rot])

## Optimization to find the calibration matrices

In [13]:
# set limits for search space
bounds = []
pos_bounds = [-0.5, 0.5]
rot_bounds = [-1, 1]
initial_guess = random_transforms(pos_bounds, rot_bounds, 2)

In [16]:
t0 = time.time()
result = minimize(evaluate_calibration, initial_guess, args=(tf_robot, tf_vrep, ))
print time.time()-t0, "seconds of optimization"

11.3935930729 seconds of optimization


In [17]:
print result

   status: 2
  success: False
     njev: 113
     nfev: 1820
 hess_inv: array([[  2.76688089e-06,  -1.33834461e-06,   4.28083207e-06,
          1.02149076e-06,   5.17668743e-07,   7.29589403e-07,
         -4.92678462e-06,   1.68719289e-07,   2.81248433e-06,
         -5.18161079e-07,  -5.99462343e-04,  -2.52031099e-04,
          8.66779867e-05,   6.24319307e-05],
       [ -1.33834461e-06,   9.93592069e-07,  -2.58502006e-06,
         -6.15326257e-07,  -2.63804289e-07,  -6.03643903e-07,
          3.11018046e-06,  -1.95088794e-07,  -1.58967597e-06,
          3.04017937e-07,   3.58706609e-04,   1.52224553e-04,
         -5.25851395e-05,  -3.81539385e-05],
       [  4.28083207e-06,  -2.58502006e-06,   7.96499975e-06,
          1.94476616e-06,   9.71085034e-07,   1.55762675e-06,
         -8.82469178e-06,   6.11093629e-07,   4.79392247e-06,
         -9.32269753e-07,  -1.06825545e-03,  -4.62546766e-04,
          1.54809412e-04,   1.11533651e-04],
       [  1.02149076e-06,  -6.15326257e-07,   1.9

## Dumping the calibration matrices

In [18]:
list_calibr = extract_transforms(result.x)

In [19]:
A = [list_calibr[0][0].tolist(), list_calibr[0][1].tolist()]
B = [list_calibr[1][0].tolist(), list_calibr[1][1].tolist()]

In [20]:
rospy.set_param("/vrep/calibration_matrix", A)

In [22]:
with open(rospack.get_path("vrep_ik_bridge")+"/config/calibration_vrep.yaml", 'w') as f:
    yaml.dump(A, f)

In [23]:
with open(rospack.get_path("vrep_ik_bridge")+"/config/calibration_tip.yaml", 'w') as f:
    yaml.dump(B, f)